In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/My Drive/practicle_projects/credit_card_fraud_detection"

/content/drive/My Drive/practicle_projects/credit_card_fraud_detection


### dataset link :- https://www.kaggle.com/mlg-ulb/creditcardfraud

In [3]:
import pandas as pd
import numpy as np
import tensorflow

np.random.seed(2)

In [4]:
# load dataset 
fraud_df = pd.read_csv("project/Dataset/creditcard.csv")
print(f"dataset shape :- \n{fraud_df.shape}")
print(f"top 5 rows :- \n {fraud_df.head()}")
print(f"columns/features names :- \n {fraud_df.columns}")
print(f"number of samples for each class :- \n {fraud_df['Class'].value_counts()}")

dataset shape :- 
(284807, 31)
top 5 rows :- 
    Time        V1        V2        V3  ...       V27       V28  Amount  Class
0   0.0 -1.359807 -0.072781  2.536347  ...  0.133558 -0.021053  149.62      0
1   0.0  1.191857  0.266151  0.166480  ... -0.008983  0.014724    2.69      0
2   1.0 -1.358354 -1.340163  1.773209  ... -0.055353 -0.059752  378.66      0
3   1.0 -0.966272 -0.185226  1.792993  ...  0.062723  0.061458  123.50      0
4   2.0 -1.158233  0.877737  1.548718  ...  0.219422  0.215153   69.99      0

[5 rows x 31 columns]
columns/features names :- 
 Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')
number of samples for each class :- 
 0    284315
1       492
Name: Class, dtype: int64


In [5]:
# make sure which features are useful & which are not
# we can remove irrelavent features 
fraud_df = fraud_df.drop(['Time'], axis=1)
print(f"list of feature names after removing Time column :- \n{fraud_df.columns}")
print(f"Info :- \n{fraud_df.info()}")

list of feature names after removing Time column :- 
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284807 non-null  float64
 1   V2      284807 non-null  float64
 2   V3      284807 non-null  float64
 3   V4      284807 non-null  float64
 4   V5      284807 non-null  float64
 5   V6      284807 non-null  float64
 6   V7      284807 non-null  float64
 7   V8      284807 non-null  float64
 8   V9      284807 non-null  float64
 9   V10     284807 non-null  float64
 10  V11     284807 non-null  float64
 11  V12     284807 non-null  float64
 12  V13     284807 non-null  float64
 13  V14     284

In [6]:
print(f"few values of Amount column :- \n {fraud_df['Amount'][0:4]}")
# data preprocessing
from sklearn.preprocessing import StandardScaler
fraud_df['norm_amount'] = StandardScaler().fit_transform(fraud_df['Amount'].values.reshape(-1,1))
fraud_df = fraud_df.drop(['Amount'], axis=1)
print(f"few values of Amount column after applying StandardScaler:- \n {fraud_df['norm_amount'][0:4]}")

few values of Amount column :- 
 0    149.62
1      2.69
2    378.66
3    123.50
Name: Amount, dtype: float64
few values of Amount column after applying StandardScaler:- 
 0    0.244964
1   -0.342475
2    1.160686
3    0.140534
Name: norm_amount, dtype: float64


In [7]:
fraud_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class,norm_amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


In [8]:
X = fraud_df.drop(['Class'], axis=1)
y = fraud_df[['Class']]

In [9]:
# spliting dataset to train & test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(199364, 29)
(85443, 29)
(199364, 1)
(85443, 1)


In [10]:
# convert Dataframe to numpy arrays
X_train_1 = np.array(X_train)
X_test_1 = np.array(X_test)
y_train_1 = np.array(y_train)
y_test_1 = np.array(y_test)

### Start model training

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
# build mode
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

def decision_tree_classification(X_train, y_train, X_test, y_test):
  # initialize object for DecisionTreeClassifier class
  dt_classifier = DecisionTreeClassifier()
  # train model by using fit method
  print("Model training start........")
  dt_classifier.fit(X_train, y_train.values.ravel())
  print("Model training completed")
  acc_score = dt_classifier.score(X_test, y_test)
  print(f'Accuracy of model on test dataset :- {acc_score}')
  # predict result using test dataset
  y_pred = dt_classifier.predict(X_test)
  # confusin matrix
  print(f"Confusion Matrix :- \n {confusion_matrix(y_test, y_pred)}")
  # classification report for f1-score 
  print(f"Classification Report :- \n {classification_report(y_test, y_pred)}")
  print(f"AROC score :- \n {roc_auc_score(y_test, y_pred)}")

def random_forest_classifier(X_train, y_train, X_test, y_test):
  # initialize object for DecisionTreeClassifier class
  rf_classifier = RandomForestClassifier(n_estimators=100)
  # train model by using fit method
  print("Model training start........")
  rf_classifier.fit(X_train, y_train.values.ravel())
  acc_score = rf_classifier.score(X_test, y_test)
  print(f'Accuracy of model on test dataset :- {acc_score}')
  # predict result using test dataset
  y_pred = rf_classifier.predict(X_test)
  # confusin matrix
  print(f"Confusion Matrix :- \n {confusion_matrix(y_test, y_pred)}")
  # classification report for f1-score 
  print(f"Classification Report :- \n {classification_report(y_test, y_pred)}")
  # area under roc curve
  print(f"AROC score :- \n {roc_auc_score(y_test, y_pred)}")


def nn_model(X_train, y_train, X_test, y_test):
  model = Sequential([
                      Dense(units = 32, input_dim = 29,activation='relu'),
                      Dense(24, activation='relu'),
                      Dropout(0.5),
                      Dense(32, activation='relu'),
                      Dense(24, activation='relu'),
                      Dense(1,activation='sigmoid')
                      ])
  print(model.summary())
  # train model
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  # model training
  model.fit(X_train, y_train, batch_size=15, epochs=5)
  # model evaluation
  performance_score = model.evaluate(X_test, y_test)
  print(f"Performance score :- \n {performance_score}")
  
  # prediction
  y_predict = model.predict_classes(X_test)
  print(f"Area Under ROC curve values :- \n {roc_auc_score(y_test, y_predict)}")
  print(f"Confusion matrix :- \n {confusion_matrix(y_test, y_predict)}")
  print(f"Classification report :- \n {classification_report(y_test, y_predict)}")

## decision tree 

In [12]:
decision_tree_classification(X_train, y_train, X_test, y_test)

Model training start........
Model training completed
Accuracy of model on test dataset :- 0.9992977774656788
Confusion Matrix :- 
 [[85273    23]
 [   37   110]]
Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.83      0.75      0.79       147

    accuracy                           1.00     85443
   macro avg       0.91      0.87      0.89     85443
weighted avg       1.00      1.00      1.00     85443

AROC score :- 
 0.8740148352531784


## Random forest tree classifier 

In [13]:
random_forest_classifier(X_train, y_train, X_test, y_test)

Model training start........
Accuracy of model on test dataset :- 0.9995435553526912
Confusion Matrix :- 
 [[85290     6]
 [   33   114]]
Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.95      0.78      0.85       147

    accuracy                           1.00     85443
   macro avg       0.97      0.89      0.93     85443
weighted avg       1.00      1.00      1.00     85443

AROC score :- 
 0.8877199304032249


## neural networks training

In [14]:
nn_model(X_train, y_train, X_test, y_test)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                792       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                800       
_________________________________________________________________
dense_3 (Dense)              (None, 24)                792       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25        
Total params: 3,369
Trainable params: 3,369
Non-trainable params: 0
______________________________________________________

#### under sampling techniques

In [15]:
class_val = fraud_df['Class'].value_counts()
print(f"Number of samples for each class :- \n {class_val}")
non_fraud = class_val[0]
fraud = class_val[1]
print(f"Non Fraudlent Numbers :- {non_fraud}")
print(f"Fraudlent Numbers :- {fraud}")

Number of samples for each class :- 
 0    284315
1       492
Name: Class, dtype: int64
Non Fraudlent Numbers :- 284315
Fraudlent Numbers :- 492


In [16]:
# take indexes of non fraudlent 
nonfraud_indexies = fraud_df[fraud_df.Class == 0].index
fraud_indices = np.array(fraud_df[fraud_df['Class'] == 1].index)
# take random samples from non fraudlent that are equal to fraudlent samples
random_normal_indexies = np.random.choice(nonfraud_indexies, fraud, replace=False)
random_normal_indexies = np.array(random_normal_indexies)

# concatenate both indecies of fraud and non fraud
under_sample_indices = np.concatenate([fraud_indices, random_normal_indexies])

#extract all features from whole data for under sample indecies only
under_sample_data = fraud_df.iloc[under_sample_indices, :]

# now we have to divide under sampling data to all features & target 
x_undersample_data = under_sample_data.drop(['Class'], axis=1)
y_undersample_data = under_sample_data[['Class']]
# now split dataset to train and test datasets as before 
X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(x_undersample_data, y_undersample_data, test_size=0.2, random_state=0)

## Decision tree with sampling data

In [17]:
decision_tree_classification(X_train_sample, y_train_sample, X_test_sample, y_test_sample)

Model training start........
Model training completed
Accuracy of model on test dataset :- 0.8984771573604061
Confusion Matrix :- 
 [[91 15]
 [ 5 86]]
Classification Report :- 
               precision    recall  f1-score   support

           0       0.95      0.86      0.90       106
           1       0.85      0.95      0.90        91

    accuracy                           0.90       197
   macro avg       0.90      0.90      0.90       197
weighted avg       0.90      0.90      0.90       197

AROC score :- 
 0.9017727555463404


## Random Forest Tree with sampling data

In [18]:
random_forest_classifier(X_train_sample, y_train_sample, X_test_sample, y_test_sample)

Model training start........
Accuracy of model on test dataset :- 0.9441624365482234
Confusion Matrix :- 
 [[99  7]
 [ 4 87]]
Classification Report :- 
               precision    recall  f1-score   support

           0       0.96      0.93      0.95       106
           1       0.93      0.96      0.94        91

    accuracy                           0.94       197
   macro avg       0.94      0.95      0.94       197
weighted avg       0.94      0.94      0.94       197

AROC score :- 
 0.9450031100974497


## Neural networks with sampling data

In [20]:
nn_model(X_train_sample, y_train_sample, X_test_sample, y_test_sample)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                960       
_________________________________________________________________
dense_11 (Dense)             (None, 24)                792       
_________________________________________________________________
dropout_2 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                800       
_________________________________________________________________
dense_13 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 25        
Total params: 3,369
Trainable params: 3,369
Non-trainable params: 0
____________________________________________________

### smote for over sqampling technique

In [21]:
!pip install imbalanced-learn

In [22]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [23]:
X_over_sample, y_over_sample = SMOTE().fit_sample(X, y.values.ravel())

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [24]:
# create data frames
X_over_sample = pd.DataFrame(X_over_sample)
y_over_sample = pd.DataFrame(y_over_sample)

In [25]:
# split dataset 
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over_sample, y_over_sample, test_size=0.3, random_state=0)

## decision tree with oversampling technique

In [26]:
decision_tree_classification(X_train_over, y_train_over, X_test_over, y_test_over)

Model training start........
Model training completed
Accuracy of model on test dataset :- 0.9980479397850975
Confusion Matrix :- 
 [[84935   237]
 [   96 85321]]
Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     85172
           1       1.00      1.00      1.00     85417

    accuracy                           1.00    170589
   macro avg       1.00      1.00      1.00    170589
weighted avg       1.00      1.00      1.00    170589

AROC score :- 
 0.9980467486675375


## random forest tree

In [27]:
random_forest_classifier(X_train_over, y_train_over, X_test_over, y_test_over)

Model training start........
Accuracy of model on test dataset :- 0.9998768971035648
Confusion Matrix :- 
 [[85151    21]
 [    0 85417]]
Classification Report :- 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     85172
           1       1.00      1.00      1.00     85417

    accuracy                           1.00    170589
   macro avg       1.00      1.00      1.00    170589
weighted avg       1.00      1.00      1.00    170589

AROC score :- 
 0.9998767200488423


#### nn

In [28]:
nn_model(X_train_over, y_train_over, X_test_over, y_test_over)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 32)                960       
_________________________________________________________________
dense_16 (Dense)             (None, 24)                792       
_________________________________________________________________
dropout_3 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 32)                800       
_________________________________________________________________
dense_18 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 25        
Total params: 3,369
Trainable params: 3,369
Non-trainable params: 0
____________________________________________________